## Loading and cleaning

In [57]:
import pandas as pd
import numpy as np
import re

In [58]:
f = open('../repository/RAW/FI - Economic Freedom.csv', mode='r', encoding='utf-8')
df = pd.read_csv(f, sep=';', index_col=False, decimal=',')
df = df.dropna(axis=0).convert_dtypes()

In [59]:
ptrn = re.compile(r'^\d\w+\s') # Removing prefixes
rm_prefix = lambda x: ptrn.sub('', x).strip()
df = df.rename(rm_prefix, axis='columns')

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 533 entries, 0 to 824
Data columns (total 85 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Year                                                         533 non-null    Int64  
 1   ISO Code 2                                                   533 non-null    string 
 2   ISO Code 3                                                   533 non-null    string 
 3   Countries                                                    533 non-null    string 
 4   Economic Freedom Summary Index                               533 non-null    Float64
 5   Rank                                                         533 non-null    Int64  
 6   Quartile                                                     533 non-null    Int64  
 7   Government consumption                                       533 non-null    Float64
